# Redis tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [Redis commands](https://redis.io/commands)
* [Redis python client](https://redis-py.readthedocs.io/en/latest/)

Though the tutorial asks you to use Redis commands, they all have an equivalent in the Python client library.

### Import libraries

In [ ]:
import redis

In [ ]:
r = redis.Redis(host='localhost', port=6379)

In [ ]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

You can launch a terminal aside, connect to your server with a Redis client and check that the value is still present :

```bash
vagrant@nosql:~$ redis-cli
127.0.0.1:6379> get foo
"bar"
```

## I. Quick start

### Strings and numbers

**Q** : Store the string `hello world` in key `greetings` with a `SET` command. Then use a `GET` command to retrieve it.

In [ ]:
r.set('greetings', 'hello world')

print(r.get('greetings'))

**Q** : Create a `connections` key with value 0. Find the `INCR` command to increment its value, then display it. 

Try to increment the value for `greetings`.

In [ ]:
r.set('connections', 0)
print('before',r.get('connections'))
r.incr('connections')

print('after',r.get('connections'))


### Lists

**Q** : Create a `colors` list with strings `red`, `black` and `blue`, using the `RPUSH` command.

In [ ]:
colors = ['red',
          'black',
          'blue']

for color in colors:
    r.rpush('colors', color)


**Q** : Display the second element of the list with `LINDEX`

In [ ]:
print(r.lindex('colors', 0)) # First element
print(r.lindex('colors', 1)) # Second element
print(r.lindex('colors', 2))  # Third element
print(r.lindex('colors', 3)) # Fourth element this case is None
print(r.lindex('colors', -1)) # Last element
print(r.lindex('colors', -2)) # Second last element

**Q** : Display the full list with `LRANGE`

In [ ]:
print("Full list:", r.lrange('colors', 0, -1))


### Using sets

**Q** : Create a `nosql` set containing values `redis`, `mongodb` and `orientdb`, using the `SADD` command.

In [ ]:
nosql = ('redis',
        'mongodb',
        'orinetndb')

for db in nosql:
    r.sadd('nosql', db)

**Q** : Test if `mysql` is inside the set, using `SISMEMBER`

In [ ]:
print(r.sismember('nosql', 'mongodb')) # Returns True or 1
print(r.sismember('nosql', 'mysql')) # Returns False or 0

**Q** : Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`. Try to add `redis` again and see what happens.

In [ ]:
r.sadd('nosql', 'hbase')

print(r.smembers('nosql'))

try:
    r.sadd('nosql', 'redis')
    print(r.smembers('nosql')) # Returns all elements in the set redis is not repeated
except Exception as e:
    print("Error:", e)

### Using sorted sets

Sorted Sets are similar to Redis Sets with the unique feature of values stored in a set. The difference is, every member of a Sorted Set is associated with a score, that is used in order to take the sorted set ordered, from the smallest to the greatest score.

**Q** : Using `ZADD`, create a sorted set `top14` with the following entries :

```
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
```

In [ ]:
city_scores = {
 'Clermont': 38,
 'Grenoble': 24,
 'La Rochelle': 26,
 'Montpellier': 32,
 'Oyonnax': 14,
 'Pau': 20,
 'Racing': 40,
 'Stade Français': 22,
 'Bordeaux': 33,
 'Castres': 29,
 'Agen': 10,
 'Toulon': 36,
 'Brive': 32,
 'Toulouse': 36,
}

for city, score in city_scores.items():
    r.zadd('city_scores', {city: score})
    


In [ ]:
print(r.zrange('city_scores', 0, -1, withscores=True)) # Returns all elements in the sorted set

**Q** : Fetch the score for `Toulon` with `ZSCORE`, and its ranking with `ZRANK`.

In [ ]:
print(r.zscore('city_scores', 'Toulon')) # Returns the score of the city
print(r.zrank('city_scores', 'Toulon')) # Returns the rank of the city

**Q** : `ZRANK` starts at 0 and scores are sorted from lowest to highest, so we should use the `ZREVRANK` for a true ranking of our cities.

In [ ]:
# get revert Rank using ZREVRANK

print(r.zrevrank('city_scores', 'Toulon')) # Returns the rank of the city

**Q** : Find the commands to display :
* the 3 best teams
* teams with more than 35 points

In [ ]:
# Get best 3 items
print(r.zrange('city_scores', 0, 2, withscores=True)) # Returns the rank of the city ascending order
print(r.zrevrange('city_scores', 0, 2, withscores=True)) # Returns the rank of the city descending order 

### Using dictionaries

**Q** : Create a dictionary `user:1` with `HMSET` with properties `id (1), name (Jean), age (22)`. Display it with `HGETALL`

In [ ]:
r.hset('user', 'id', '1')
r.hset('user', 'name', 'Jean')
r.hset('user', 'age', 29)



In [ ]:
r.hgetall('user')

**Q** : Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

In [ ]:
r.hset('user', 'city', 'Lyon') # New attribute
r.hset('user', 'name', 'Juan') # Update the name

print(r.hgetall('user'))


## Modelling a query cache with Redis

You are modeling data from a REST query cache system with Redis.
A request is identified by the http method and its url (without the protocol).
The content of the request is stored as it is to be returned on demand.

**Q** : Insert a PUT request in the cache on http://my-api.fr/user/10 whose answer is {"id": 10, "name": "jean"}

**SOLUTION:** 
<br>
Start API using uvicorn and command `uvicorn app:app --host 0.0.0.0 --port 8000 --reload`


In [ ]:
# Create the cache key
cache_key = "PUT:my-api.fr/user/10"

# Content
response_content = '{"id": 10, "name": "jean"}'

# Store in Redis cache
r.set(cache_key, response_content)

# Verify the content was stored correctly
print("Stored content:", r.get(cache_key))

**Q** : Create a set of cache request keys.

In [ ]:

keys = [cache_key,
        'GET:my-api.fr/users',
        'POST:my-api.fr/user']

for key in keys:
    r.sadd('cache_keys', key)
    
# Display all cache keys in the set
print("Cache keys:", r.smembers('cache_keys'))

**Q** : Check if the GET request on http: //http://my-api.fr/user

In [ ]:
# Create the key to check
check_key = 'GET:my-api.fr/users'

# Using sismember check if exists
is_in_cache = r.sismember('cache_keys', check_key)

print(f"Checking if {check_key} exists in cache")
print(is_in_cache)

# Check if the key exists and has content
if is_in_cache:
    content = r.get(check_key)
    print(f"Request found in cache with content: {content}")
else:
    print("Request not found in cache")

**Q** : Delete the PUT request on http://my-api.fr/user/10 from the cache.

In [ ]:
key_to_delete = cache_key

# Delete the key from the set
r.srem('cache_keys', key_to_delete)

# Verify the key was removed
print("Cache keys after deletion:", r.smembers('cache_keys'))


## Postquisites

The folloinwg command removes all data from your Redis cluster.

In [ ]:
!redis-cli flushall